## W205 Project 2: Pyspark Cluster
#### Author: Kevin Xuan

Now we have entered the pyspark container. Inside this container, we will flatten the messy data ainto a structured table and remove unncessary data so that we or other data scientists can conduct further data analysis on the data and do not have to worry about dealing with unstructured or unnucessary data. First we will read data which is stored in `student-asssessment`.

### Read and Understand Data

In [1]:
# Read data that is stored in the topic student-assessment
data = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:29092") \
  .option("subscribe","student-assessment") \
  .option("startingOffsets", "earliest") \
  .option("endingOffsets", "latest") \
  .load() 

To understand the command, we again are retrieving data from other containers, so we need to connect to kafka again with the `29092` port number. Since data is stored in kafka topic `student-assessment`, we have to specify that in our parameter. Next, we want to sort data based on time, so we specify startingoffsets and endingoffsets.

In [5]:
# Cache this to Remove Warning messages which will appear later
data.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [14]:
# Print Schema for this dataset
data.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [4]:
# Display the first 5 rows for the data
data.show(5)

+----+--------------------+------------------+---------+------+--------------------+-------------+
| key|               value|             topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------------------+---------+------+--------------------+-------------+
|null|[7B 22 6B 65 65 6...|student-assessment|        0|     0|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|student-assessment|        0|     1|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|student-assessment|        0|     2|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|student-assessment|        0|     3|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|student-assessment|        0|     4|1969-12-31 23:59:...|            0|
+----+--------------------+------------------+---------+------+--------------------+-------------+
only showing top 5 rows



We are only interested in the value column, so we select the "value" column. We see that "value" column is encoded, so we need to convert the column to string format so that the texts are readable. To do so, we perform the command below:

In [6]:
# Convert "value" column to string format
exam_data = data.select(data.value.cast('string'))

In [ ]:
# Display the first 5 rows of the data
exam_data.show(5)

After converting the "value" column to string format, we see element in each row is a dictionary containing many keys and values. To make the data look better and into a structured table with many columns, for each row we load the dictionary and convert to a dataframe.

### Convert json column to dataframe

In [19]:
# import json python package
import json

/spark-2.2.0-bin-hadoop2.6/python/pyspark/sql/session.py:351: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [ ]:
# We see that each row is a dictionary, so for each row we load the dictionary using json package and convert to dataframe
exam = exam_data.rdd.map(lambda x: json.loads(x.value)).toDF()

In [ ]:
# Display first 5 rows of the data
extracted_exam.show(5)

In [20]:
# print the schema of the new dataframe
exam.printSchema()

root
 |-- base_exam_id: string (nullable = true)
 |-- certification: string (nullable = true)
 |-- exam_name: string (nullable = true)
 |-- keen_created_at: string (nullable = true)
 |-- keen_id: string (nullable = true)
 |-- keen_timestamp: string (nullable = true)
 |-- max_attempts: string (nullable = true)
 |-- sequences: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: map (containsNull = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: boolean (valueContainsNull = true)
 |-- started_at: string (nullable = true)
 |-- user_exam_id: string (nullable = true)



In [ ]:
# We see that all the columns except "sequences" 

In [29]:
extracted_exam.select('sequences').collect()[0]

Row(sequences={'questions': [{'options': None, 'user_correct': False, 'user_incomplete': True, 'id': None, 'user_result': None, 'user_submitted': True}, {'options': None, 'user_correct': False, 'user_incomplete': False, 'id': None, 'user_result': None, 'user_submitted': True}, {'options': None, 'user_correct': True, 'user_incomplete': False, 'id': None, 'user_result': None, 'user_submitted': True}, {'options': None, 'user_correct': True, 'user_incomplete': False, 'id': None, 'user_result': None, 'user_submitted': True}], 'id': None, 'attempt': None, 'counts': None})

In [ ]:
# This data does not help answer our questions. 

In [ ]:
# save table to HDFS
# ad.write.parquet("/tmp/exam_data")

In [33]:
# Since "sequence" column does not help answer our business questions, we can remove it
extracted_exam = extracted_exam.drop('sequences')

In [61]:
extracted_exam.printSchema()

root
 |-- base_exam_id: string (nullable = true)
 |-- certification: string (nullable = true)
 |-- exam_name: string (nullable = true)
 |-- keen_created_at: string (nullable = true)
 |-- keen_id: string (nullable = true)
 |-- keen_timestamp: string (nullable = true)
 |-- max_attempts: string (nullable = true)
 |-- started_at: string (nullable = true)
 |-- user_exam_id: string (nullable = true)



In [34]:
extracted_exam.show(5)

+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|   keen_created_at|             keen_id|    keen_timestamp|max_attempts|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717442.735266|5a6745820eb8ab000...| 1516717442.735266|         1.0|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717377.639827|5a674541ab6b0a000...| 1516717377.639827|         1.0|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false|The Principles of...| 1516738973.653394|5a67999d3ed3e3000...| 1516738973.653394|         1.0|2018-01-23T

In [ ]:
# Land table to HDFS
extracted_exam.write.parquet("/tmp/extracted_exam")

In [38]:
# Register a temporary view for the table "extracted_exam"
extracted_exam.registerTempTable('exam_data')

# Questions

Question 1: How many assessments are in the dataset?

In [55]:
spark.sql('''
    select count(distinct(user_exam_id)) as num_of_assessments
    from exam_data'''
    ).show()

+------------------+
|num_of_assessments|
+------------------+
|              3242|
+------------------+



Answer: There are 3,242 assessments in the dataset.

Question 2: What's the name of your Kafka topic? How did you come up with that name?

In [ ]:
Answer: 

Question 3: How many people took Learning Git?

In [57]:
spark.sql('''
    select count(distinct(user_exam_id)) as num_of_people_took_Learning_Git
    from exam_data
    where exam_name = 'Learning Git'
    ''').show()

+-------------+
|num_of_people|
+-------------+
|          390|
+-------------+



Answer: 390 people took Learning Git.

In [ ]:
Question 4: What is the least common course taken? And the most common?

In [59]:
# least common course taken
spark.sql('''
    select exam_name, count(distinct(user_exam_id)) as num_of_students
    from exam_data
    group by exam_name
    order by count(exam_name)
    limit 1
    ''').collect()

[Row(exam_name='Learning to Visualize Data with D3.js', num_of_students=1)]

In [60]:
# most common course
# use collect to show entire name of the field
spark.sql('''
    select exam_name, count(distinct(user_exam_id)) as num_of_students
    from exam_data
    group by exam_name
    order by count(exam_name) DESC
     limit 1
    ''').collect()

[Row(exam_name='Learning Git', num_of_students=390)]

Answer: The least common course taken is "Learning to Visualize Data with D3.js". The most common course taken is "Learning Git".

In [ ]:
# 4. What is the least common course taken? And the most common?
# Answer: The least common course taken is "Learning to Visualize Data with D3.js". 
#         The most common course taken is "Learning Git"

In [ ]:
# 5. How many courses are there in the dataset?

In [ ]:
spark.sql(\
    "select count(distinct exam_name) from exam_data").show()

In [ ]:
# 6. What's the earliest and latest date for an exam?
spark.sql(\
    "select min(started_at) as earliest_date, max(started_at) as latest_date from exam_data").show()

In [ ]:
# The earliest exam is on Novemember 21st in 2017, and the latest exam is January 28th on 2018.

In [ ]:
spark.sql(\
    "select extract(hour from started_at) from exam_data").show()


In [ ]:
# 7. 